# BRISQUE score

## single image

In [ ]:
#BRISQUE
#Blind/Referenceless Image Spatial Quality Evaluator
#Single number score for image quality (to facilitate comparing photos and selecting 'best' photo)

'''
add to requirements.txt !!!!!!! _> #pip install brisque
'''
#pip install brisque

from PIL import Image
import numpy as np
from brisque import BRISQUE

# Load the image using PIL
image_path = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/DSC_0797.JPG'
image = Image.open(image_path)

# Convert the image to RGB (if it's not already)
image = image.convert('RGB')

# Convert the image to a numpy array
image_array = np.array(image)

# Calculate the BRISQUE score
brisque_score = BRISQUE().score(image_array)

print(f"BRISQUE score for the image is: {brisque_score}")

#For Image via URL :
#obj = BRISQUE(url=True).score("https://images.ctfassets.net/u4vv676b8z52/4dy4qSI9G77nmLayGrUl73/08fdde4d35a717c7e9d123ef2a478e9b/blurred-vision-678x446_compressed.jpg?fm=jpg&q=80")


## BRISQUE score for multiple images in a folder (with error avoidance code)

In [ ]:
import os
from PIL import Image
import numpy as np
from brisque import BRISQUE
import pandas as pd

# Function to process a single image
def process_image(image_file):
    # Construct the full image path
    image_path = os.path.join(image_dir, image_file)

    # Check if the path is a file and is a recognized image format
    if os.path.isfile(image_path) and image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        try:
            # Load the image using PIL
            image = Image.open(image_path)

            # Convert the image to RGB (if it's not already)
            image = image.convert('RGB')

            # Resize the image
            image = image.resize((256, 256))

            # Convert the image to a numpy array
            image_array = np.array(image)

            # Calculate the BRISQUE score
            brisque_score = BRISQUE().score(image_array)

            return (image_file, brisque_score)
        except Exception as e:
            print(f"Error processing file {image_file}: {e}")

# Directory containing the images
image_dir = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/"

# List all the files in the directory
image_files = os.listdir(image_dir)

# Initialize a list to store the results
results = []

# Process each image file
for image_file in image_files:
    result = process_image(image_file)
    if result is not None:
        results.append(result)

# Convert the results to a pandas DataFrame
df = pd.DataFrame(results, columns=['Image', 'BRISQUE Score'])

# Display the DataFrame
print(df)


In [ ]:
df.to_csv('/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/brisque_scores2.csv')

# Tiling for Deblur model

In [1]:
'''
Model to deblur images, using EDSR.
takes image from EDI
    nparr = np.frombuffer(contents, np.uint8) # convert string data to np array
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR) # convert np array to image
if the input image is larger than 768x768 then it scales the image down so longest side is 768 pixels,
then pads short side to reach 768 pixels,
then splits padded image into 9 tiles,
process each tile individually,
then reconstruct the enhanced image from the enhanced patches
and remove padding from the enhanced image.
'''

import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2

# initial model_path = '/home/gregrevill/code/GregRevFr/photopocalypse/models/SRresnet-from-kaggle-trained-on-blur.h5'
# paulo model model_path = '/home/gregrevill/code/GregRevFr/photopocalypse/models/EDSR_blur_dataset_model_2.h5'
model_path = '/home/gregrevill/code/GregRevFr/photopocalypse/models/gen_model3000.h5'
image_path = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/DSC_0797.JPG'
model = load_model(model_path)
tile_size = 256
number_of_tiles = 9

def deblur_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Resize if larger than 768x768
    max_side = tile_size * int(number_of_tiles ** 0.5)
    scale_factor = min(max_side / image.shape[0], max_side / image.shape[1])
    if scale_factor < 1:
        new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
        image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

    # Padding
    original_height, original_width = image.shape[:2]
    pad_height = (tile_size - image.shape[0] % tile_size) % tile_size
    pad_width = (tile_size - image.shape[1] % tile_size) % tile_size
    image = cv2.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv2.BORDER_CONSTANT, value=0)

    # Split into tiles
    tiles = []
    for i in range(0, image.shape[0], tile_size):
        for j in range(0, image.shape[1], tile_size):
            tile = image[i:i + tile_size, j:j + tile_size]
            tiles.append(tile)

    # Predict and enhance each tile
    # skip predict step for now
    # enhanced_tiles = [model.predict(np.expand_dims(tile, 0))[0] for tile in tiles]
    enhanced_tiles = [(model.predict(np.expand_dims(tile / 255., 0)) * 255)[0].astype(np.uint8) for tile in tiles]
    # enhanced_tiles = tiles  # Using the original tiles for now

    # Reconstruct the enhanced image
    rows = []
    for i in range(0, len(enhanced_tiles), int(image.shape[1] / tile_size)):
        rows.append(np.hstack(enhanced_tiles[i:i + int(image.shape[1] / tile_size)]))
    enhanced_image = np.vstack(rows)

    # Calculate the actual enhanced image dimensions after padding
    actual_enhanced_height = original_height + pad_height
    actual_enhanced_width = original_width + pad_width

    # Remove padding
    enhanced_image = enhanced_image[:actual_enhanced_height, :actual_enhanced_width]

    # Now crop back to original size
    enhanced_image = enhanced_image[:original_height, :original_width]

    return enhanced_image

input_folder = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/'
output_folder = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/output_images_modelv2/'

# Get list of image file paths from the input directory
image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Process each image
for file_name in image_files:
    image_path = os.path.join(input_folder, file_name)
    enhanced_image = deblur_image(image_path)

    # Create a new filename with 'deblurred' appended
    new_filename = f"deblurred_{file_name}"
    output_path = os.path.join(output_folder, new_filename)

    # Save the enhanced image
    cv2.imwrite(output_path, enhanced_image)


2023-12-07 10:11:47.540523: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# display original image and super resolution image side by side with labels

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the images
img1 = mpimg.imread(image_path)
img2 = mpimg.imread(enhanced_image)

# Create a figure with two subplots
fig, ax = plt.subplots(1, 2)

# Display the images
ax[0].imshow(img1)
ax[0].set_title('Original Image')
ax[0].axis('off')

ax[1].imshow(img2)
ax[1].set_title('Super Resolution Image')
ax[1].axis('off')

# Show the figure
plt.show()


In [8]:
import cv2

def resize_and_save_image(image_path, output_folder, sizes):
    """
    Resizes an image to multiple sizes and saves them.

    :param image_path: Path to the original image.
    :param output_folder: Folder where resized images will be saved.
    :param sizes: List of tuple sizes to resize the image to. Example: [(800, 600), (1024, 768)]
    """
    # Read the original image
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Image not found.")
        return

    for size in sizes:
        # Resize the image
        resized_image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)

        # Construct the output file path
        output_path = f"{output_folder}/2_{size[0]}x{size[1]}.jpg"

        # Save the resized image
        cv2.imwrite(output_path, resized_image)
        print(f"Image saved to {output_path}")

# Example usage
image_path = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/Photopocolypse_WBG.png'
output_folder = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/"  # Replace with your desired output folder
sizes = [(600, 600)]  # Different sizes for testing

resize_and_save_image(image_path, output_folder, sizes)


Error: Image not found.


[ WARN:0@236.512] global loadsave.cpp:248 findDecoder imread_('/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/Photopocolypse_WBG.png'): can't open/read file: check file path/integrity


In [11]:
# resizes folder of images to 600x600

import os
import cv2

def resize_image(image_path, output_dir, size=(600, 600)):
    if not os.path.isfile(image_path) or not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        return

    try:
        image = cv2.imread(image_path)
        height, width, channels = image.shape

        if height > width:
            image = image[0:width, 0:width] # crops from the bottom to make square

        resized_image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
        output_path = os.path.join(output_dir, os.path.basename(image_path))
        cv2.imwrite(output_path, resized_image)
    except Exception as e:
        print(f"Error processing file {image_path}: {e}")

image_dir = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/"
output_dir = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/600x600/"

os.makedirs(output_dir, exist_ok=True)

for image_file in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_file)
    resize_image(image_path, output_dir)

In [5]:
# RealESRGAN model

import torch
from PIL import Image
import numpy as np
# pip install realesrgan
from RealESRGAN import RealESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

path_to_image = 'inputs/lr_image.png'
image = Image.open(path_to_image).convert('RGB')

sr_image = model.predict(image)

sr_image.save('results/sr_image.png')


ModuleNotFoundError: No module named 'RealESRGAN'